# 2024-11-15

# The purpose of this notebook submit my own ML modeling based  

#### Changes 1/3/25

Main points that increasing scores
-reset index after dropping sii rows
-data cleaning for both test and train before training the model
    -include dropping null columns 

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier #Jason first model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.impute import KNNImputer
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
#from sklearn.feature_selection import VarianceThreshold #decide to eliminate this because wont include features interaction 
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split #train the first model
from sklearn.ensemble import RandomForestClassifier #train the first model
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix #train the first model

#from sklearn.ensemble import RandomForestRegressor #get second model
from sklearn.tree import DecisionTreeClassifier # get third model

from sklearn.ensemble import GradientBoostingClassifier #this help handle imbalance classes
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob

#pd.options.display.max_rows = 100
#SEED = 101

In [ ]:
#Read train data

df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
# Select all columns except 'id'
df_except_id = df.loc[:, ~df.columns.isin(['id'])]
df_except_id.info()

In [ ]:
df_except_id.head()

In [ ]:
# Exclude rows where the 'sii' column has null values
df_except_id = df_except_id[df_except_id['sii'].notnull()]
# Drop columns contain null values
df_except_id = df_except_id.dropna(axis=1)

#reset index after dropping rows
df_except_id = df_except_id.reset_index(drop=True, inplace=False) #assign a variable to save the dataframe,inplace to False mean it will not change the orginal dataset

df_except_id.info()

In [ ]:
## code changes alignment purpose only

df_encoded = df_except_id

In [ ]:
# Function to aggregate data
def agg_data(split, id):
    fname = f"/kaggle/input/child-mind-institute-problematic-internet-use/series_{split}.parquet/id={id}/part-0.parquet"
    results = {'id': id}
    try:
        df_sub = pd.read_parquet(fname)
        # Calculate the mean and standard deviation for each column
        results['avg_step'] = df_sub['step'].mean()
        results['std_step'] = df_sub['step'].std()

        results['avg_X'] = df_sub['X'].mean()
        results['std_X'] = df_sub['X'].std()

        results['avg_Y'] = df_sub['Y'].mean()
        results['std_Y'] = df_sub['Y'].std()

        results['avg_Z'] = df_sub['Z'].mean()
        results['std_Z'] = df_sub['Z'].std()

        results['avg_enmo'] = df_sub['enmo'].mean()
        results['std_enmo'] = df_sub['enmo'].std()

        results['avg_anglez'] = df_sub['anglez'].mean()
        results['std_anglez'] = df_sub['anglez'].std()

        results['avg_non_wear_flag'] = df_sub['non-wear_flag'].mean()
        results['std_non_wear_flag'] = df_sub['non-wear_flag'].std()

        results['avg_light'] = df_sub['light'].mean()
        results['std_light'] = df_sub['light'].std()

        results['avg_battery_voltage'] = df_sub['battery_voltage'].mean()
        results['std_battery_voltage'] = df_sub['battery_voltage'].std()

        results['avg_time_of_day'] = df_sub['time_of_day'].mean()
        results['std_time_of_day'] = df_sub['time_of_day'].std()

        results['avg_weekday'] = df_sub['weekday'].mean()
        results['std_weekday'] = df_sub['weekday'].std()

        results['avg_quarter'] = df_sub['quarter'].mean()
        results['std_quarter'] = df_sub['quarter'].std()

        results['avg_relative_date_PCIAT'] = df_sub['relative_date_PCIAT'].mean()
        results['std_relative_date_PCIAT'] = df_sub['relative_date_PCIAT'].std()
        
    except FileNotFoundError:
        results['avg_step'] = 0
        results['std_step'] = 0

        results['avg_X'] = 0
        results['std_X'] = 0

        results['avg_Y'] = 0
        results['std_Y'] = 0

        results['avg_Z'] = 0
        results['std_Z'] = 0

        results['avg_enmo'] = 0
        results['std_enmo'] = 0

        results['avg_anglez'] = 0
        results['std_anglez'] = 0

        results['avg_non_wear_flag'] = 0
        results['std_non_wear_flag'] = 0

        results['avg_light'] = 0
        results['std_light'] = 0

        results['avg_battery_voltage'] = 0
        results['std_battery_voltage'] = 0

        results['avg_time_of_day'] = 0
        results['std_time_of_day'] = 0

        results['avg_weekday'] = 0
        results['std_weekday'] = 0

        results['avg_quarter'] = 0
        results['std_quarter'] = 0

        results['avg_relative_date_PCIAT'] = 0
        results['std_relative_date_PCIAT'] = 0

    return pd.Series(results)

data = []
for id in tqdm(df['id']):
    row = agg_data('train', id)
    data.append(row)

In [ ]:
df_agg = pd.DataFrame(data)
df_agg.head()

In [ ]:
# Append the 'id' column back to the transformed DataFrame
df_encoded['id'] = df['id']
df_encoded.head()

In [ ]:
df = df_encoded.merge(df_agg, on='id') 
df.head()

In [ ]:
df_test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
df_test.head()

In [ ]:
df_test.info()

In [ ]:
#### Data cleaning for TEST data

# Drop columns contain null values
df_test = df_test.dropna(axis=1)

#reset index after dropping rows
df_test = df_test.reset_index(drop=True, inplace=False) #assign a variable to save the dataframe,inplace to False mean it will not change the orginal dataset

df_test.info()

In [ ]:
#Now let append parquet data to test data
data = []
for id in tqdm(df_test['id']):
    row = agg_data('test', id)
    data.append(row)
df_test_agg = pd.DataFrame(data)
df_test = df_test.merge(df_test_agg, on='id')
df_test.head()

In [ ]:
# This turning the df into set can see no duplication of train that give you
#columns only in train not in test
test_col_set = set(df_test.columns)
train_col_set = set(df.columns)
train_col_set.difference(test_col_set)

In [ ]:
feat_cols = [col for col in df.columns if ('PCIAT' not in col) & (col != 'sii') & (col != 'id')]
feat_cols

In [ ]:
df[feat_cols].info()

In [ ]:
enc_dict = {}
for feat in feat_cols:
    if df[feat].dtype != 'int64':
        enc = LabelEncoder()
        df[feat] = enc.fit_transform(df[feat])
        enc_dict[feat] = enc

print(df[feat_cols])

In [ ]:
df[feat_cols].info()

In [ ]:
df['sii'].info()

In [ ]:
# Change the data type of the 'sii' column to int64
df['sii'] = df['sii'].astype('int64')
df['sii'].info()

In [ ]:
#TEST
enc_dict = {}
for feat in feat_cols:
    if df_test[feat].dtype != 'int64':
        enc = LabelEncoder()
        df_test[feat] = enc.fit_transform(df_test[feat])
        enc_dict[feat] = enc

In [ ]:
df[feat_cols].head(2)

In [ ]:
df_test[feat_cols].head(2)

In [ ]:
target_col = 'sii'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[feat_cols], df[target_col], test_size=0.2, random_state=42)

# Ensure the target column is categorical
df[target_col] = df[target_col].astype('int64')
df[feat_cols] = df[feat_cols].astype('category')

# Split the data into training and testing sets
X = df[feat_cols]
y = df[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute class weights (useful for imbalanced classes)
#class_weight_dict = dict(zip(np.unique(y_train), class_weights))
class_weight_dict = {0: 1.72, 1: 3.75, 2: 7.23, 3: 80.00}

# Initialize GradientBoostingClassifier
model = HistGradientBoostingClassifier(random_state=42, class_weight = class_weight_dict)

# Train the model on the training data
model.fit(X,y)

# Make predictions on the test set
y_pred = model.predict(X_test)


In [ ]:
###train data prediction accuracy without split

print('Test Accuracy: %.3f' % model.score(X_test, y_test))

In [ ]:
###train data prediction accuracy WITH split
#### WITH split, the test accuracy goes down by 0.2

#print('Test Accuracy: %.3f' % model.score(X_test, y_test))

In [ ]:
#### Cross validation to check the score again

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from numpy import mean, std

# Define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(f'Accuracy: {mean(n_scores):.3f} ({std(n_scores):.3f})')

In [ ]:
# Make predictions
df_test['sii'] = model.predict(df_test[feat_cols])

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
df_test.columns

In [ ]:
df_test['sii'] = df_test['sii'].astype(int)
df_test[['id', 'sii']].to_csv('submission.csv', index=False)